In [ ]:
#Class Project (Adam)
#*********************Predicting the helpfulness of Amazon movie product reviews
#*********************My developed classifier assesses a dataset of reviews
#*********************I will outline my approach to predicting the helpfulness of a review
#*********************the developed classifier will assess reviews randomly from the dataset 
#*********************and predict whether a given product review is either ”helpful” or ”unhelpful.” 
#*********************Will initially use reviewer/responder metrics along with star rankings of the reviewer 
#*********************relative to the polpulation - second phase  - will use text analysis on the actual
#*********************reviews as well.  
#*********************I was able to achieve up to ___ accuracy on movie reviews.
#   Data Source: Amazon movie reviews past 12 years

In [12]:
#preparing the data and creating a pandas data frame
import pandas as pd
with open(r'/Users/Adam/Documents/DataClass/project/movies_small.txt') as f:
    dict_list = []
    record_dict = {}
    for line in f:
#       print line
        if line.strip() is '':
            dict_list.append(record_dict)
            record_dict = {} # reset record_dict
        else:   
            split_values = line.split(': ',1)
            key = split_values[0]
            try:
                value = split_values[1].rstrip()
            except IndexError:
                value = ''
            record_dict[key] = value

df = pd.DataFrame(dict_list) # store in dataframe
df.head()

,product/productId,review/helpfulness,review/profileName,review/score,review/summary,review/text,review/time,review/userId
0,B003AI2VGA,7/7,"Brian E. Erland ""Rainbow Sphinx""",3.0,"""There Is So Much Darkness Now ~ Come For The ...","Synopsis: On the daily trek from Juarez, Mexic...",1182729600,A141HP4LYPWMSR
1,B003AI2VGA,4/4,Grady Harp,3.0,Worthwhile and Important Story Hampered by Poo...,THE VIRGIN OF JUAREZ is based on true events s...,1181952000,A328S9RN3U5M68
2,B003AI2VGA,8/10,"Chrissy K. McVay ""Writer""",5.0,This movie needed to be made.,The scenes in this film can be very disquietin...,1164844800,A1I7QGUDP043DG
3,B003AI2VGA,1/1,golgotha.gov,3.0,distantly based on a real tragedy,THE VIRGIN OF JUAREZ (2006)<br />directed by K...,1197158400,A1M5405JH9THP9
4,B003AI2VGA,1/1,"KerrLines ""&#34;Movies,Music,Theatre&#34;""",3.0,"""What's going on down in Juarez and shining a ...","Informationally, this SHOWTIME original is ess...",1188345600,ATXL536YX71TR


In [13]:
#df.append(df['review/helpfulness'].apply(lambda x: pd.Series(x.split('/'), index = x.index)))
df = df.join(pd.DataFrame(df['review/helpfulness'].str.split('/').tolist(), columns=['num_helpful','num_total']).astype(int))
df.drop('review/helpfulness', inplace=True, axis=1)
#df.loc[(df['num_total'] > 200)]

In [17]:
df['helpful_ratio'] = df['num_helpful']/df['num_total']
df1 = df[['review/score','num_helpful','num_total','helpful_ratio']]
df1.head()

,review/score,num_helpful,num_total,helpful_ratio
0,3.0,7,7,1.0
1,3.0,4,4,1.0
2,5.0,8,10,0.8
3,3.0,1,1,1.0
4,3.0,1,1,1.0


In [18]:
import numpy as np
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import RidgeCV
%matplotlib inline

In [19]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43390 entries, 0 to 43389
Data columns (total 4 columns):
review/score     43390 non-null object
num_helpful      43390 non-null int64
num_total        43390 non-null int64
helpful_ratio    31300 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 1.7+ MB


In [20]:
# How many rows and columns does the dataset have?
df1.shape

(43390, 4)

In [23]:
# View the correlations
df1.corr()

,num_helpful,num_total,helpful_ratio
num_helpful,1.000000,0.954179,0.137273
num_total,0.954179,1.000000,0.027192
helpful_ratio,0.137273,0.027192,1.000000
